In [91]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [92]:
csv_file = "statesdata.csv"
state_data_df = pd.read_csv(csv_file)
state_data_df.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [93]:
new_state_data_df = state_data_df[['date', 'state','cases','deaths']].copy()
new_state_data_df.head()

,date,state,cases,deaths
0,2020-01-21,Washington,1,0
1,2020-01-22,Washington,1,0
2,2020-01-23,Washington,1,0
3,2020-01-24,Illinois,1,0
4,2020-01-24,Washington,1,0


In [94]:
SW = ['California', 'Nevada', 'Utah', 'Arizona']
new_state_data_df["region"] = np.where(new_state_data_df['state'].isin(SW), 'SW','Unknown')
aggregate_data_df = new_state_data_df.loc[new_state_data_df['region'] == 'SW']
aggregate_data_df.head()

,date,state,cases,deaths,region
5,2020-01-25,California,1,0,SW
8,2020-01-26,Arizona,1,0,SW
9,2020-01-26,California,2,0,SW
12,2020-01-27,Arizona,1,0,SW
13,2020-01-27,California,2,0,SW


In [95]:
sw_data_df = aggregate_data_df[['date', 'region','cases','deaths']].copy()
sw_data_df.head()

,date,region,cases,deaths
5,2020-01-25,SW,1,0
8,2020-01-26,SW,1,0
9,2020-01-26,SW,2,0
12,2020-01-27,SW,1,0
13,2020-01-27,SW,2,0


In [96]:
sw_data_df['date'] = pd.to_datetime(sw_data_df['date']).dt.date
sw_data_df.sort_values(by='date')
print(sw_data_df)

             date region    cases  deaths
5      2020-01-25     SW        1       0
8      2020-01-26     SW        1       0
9      2020-01-26     SW        2       0
12     2020-01-27     SW        1       0
13     2020-01-27     SW        2       0
...           ...    ...      ...     ...
38022  2022-01-20     SW   814547    4019
38033  2022-01-21     SW  1701949   25502
38035  2022-01-21     SW  7621774   78700
38060  2022-01-21     SW   614505    8736
38078  2022-01-21     SW   826149    4030

[2840 rows x 4 columns]


In [97]:
final_df = sw_data_df.groupby('date').sum()
final_df.sort_values(by='date')
final_df.reset_index()

,date,cases,deaths
0,2020-01-25,1,0
1,2020-01-26,3,0
2,2020-01-27,3,0
3,2020-01-28,3,0
4,2020-01-29,3,0
...,...,...,...
723,2022-01-17,10085863,115932
724,2022-01-18,10247594,116252
725,2022-01-19,10395170,116426
726,2022-01-20,10593531,116686


In [98]:
rds_connection_string = "postgres:mochi@localhost:5432/covid_case_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [99]:
final_df.to_sql(name='case_SW', con=engine)